In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, mapping, shape
from imp import reload
from numpy import mean
from planet_utils import  search
from numpy.random import randint, choice
import random
from datetime import datetime

import folium
import json
import requests
import os
import cartopy.crs as ccrs
from cartopy import feature
#from retrying import retry
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.patches import Circle
import numpy as np
from multiprocessing.dummy import Pool as ThreadPool 

plt.rcParams['figure.figsize'] = (20,20)
%matplotlib inline

#print(os.environ["PL_API_KEY"])
NUM_RANDOM_DATES = 10
NUM_RANDOM_LOCATIONS = 20

image_utils: found API key


# API Search Candidate Selection Protocol
The goal of this notebook is to develop the pathway from a set of single-measurement points to a set of cropped PlanetScope imagery for a given date band. 

## Extract 2017 Measurement Locations

In [2]:
snowdata = pd.read_csv("../data/snow_summary_all_2009_2017_locs.csv", 
                       parse_dates = ["snow_appearance_date", "snow_disappearance_date", 
                                      "date_min", "date_max"])
snowdata = snowdata[snowdata.year >= 2017]
snowdata['geometry'] = [Point(xy) for xy in zip(snowdata.longitude, snowdata.latitude)]
snowdata = gpd.GeoDataFrame(snowdata)
snowdata.crs = {'init' : 'epsg:4326'}


In [3]:
locations = snowdata.dropna(subset=["longitude", 'latitude']).drop_duplicates("Location")



In [4]:
locations = locations.loc[choice(locations.index, NUM_RANDOM_LOCATIONS, replace=False)]


In [5]:
len(locations)

20

## Add bounding boxes

In [6]:
boxes = locations[['Location', 'geometry']].copy()
boxes.geometry = [g.buffer(0.005, cap_style=3) for g in boxes.geometry]

## Search

In [7]:
a = search.SimpleSearch(boxes.geometry.values[0], datetime(2018, 5, 1), datetime(2018, 5, 10)).query()

Querying Planet API: 100%|██████████| 1/1 [00:01<00:00,  1.96s/searches]


In [36]:
cc = ([abs(pd.to_datetime(r['acquired']) - datetime(2016, 5, 1)) for r in a.properties.values])

In [8]:
a.id


0    20180506_182751_1039
1    20180503_182846_0f52
2    20180502_182800_100a
3    20180502_182759_100a
4    20180502_182706_101d
Name: id, dtype: object

In [19]:
reload(search)
dates = locations[['Location', "snow_appearance_date", "snow_disappearance_date"]]
searcher = search.Search(boxes, dates, dry=False,
                         key='Location', start_col='snow_appearance_date',
                         end_col="snow_disappearance_date")
results = searcher.query()


Querying Planet API:   5%|▌         | 1/20 [00:13<04:21, 13.75s/searches]

KeyboardInterrupt: 

## Parse Results
Choose `NUM_RANDOM_DATES` dates from results for each loc

In [26]:
loc_img_ids = {}
for group in results.groupby('loc_id'):
    if (len(group[1]) >= NUM_RANDOM_DATES):
        loc_img_ids[group[0]] = list(set(choice(group[1].id.values, NUM_RANDOM_DATES, replace=False)))
    else:
        loc_img_ids[group[0]] = list(set(group[1].id.values))



In [73]:
CLIP_API_URL = "https://api.planet.com/compute/ops/clips/v1/"
IMAGEDIR = "../images/"
PL_API_KEY = os.environ["PL_API_KEY"]

In [92]:
def clip_request_and_download(loc, image):

    @retry(wait_fixed=5000)
    def _check_clip_op(id):
        r = requests.get("{_base}/{id}".format(_base = CLIP_API_URL, id=id), auth=(PL_API_KEY, ""))
        if r.json()['state'] != "succeeded":
            print("...waiting")
            raise Exception("Not Yet")
        else:
            print("response found.")
            return(r.json())
    
    geom = boxes.loc[loc].geometry
        
    payload = {
        "aoi" : mapping(geom),
        "targets" : [{
            "item_id" : image, 
            "item_type" : "PSScene4Band", 
            "asset_type" : 'analytic'
        }]
    }

    r = requests.post(CLIP_API_URL, auth=(PL_API_KEY, ""), json=payload)
    print(r.json())
    
    response = _check_clip_op(r.json()['id'])

    image_url = response['_links']['results'][0]
    
    local_filename = os.path.join(IMAGEDIR, "{loc}_{img}.zip".format(loc=loc, img=image))

    r = requests.get(image_url, stream=True, auth=(PL_API_KEY, ""))
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return local_filename



In [140]:
len(loc_img_ids)

83

In [113]:
reload(download)
files = {}
for loc_id, img_ids in loc_img_ids.items():
    box = boxes.loc[loc_id].geometry
    dl = download.CroppedDownload(loc_id, box, img_ids[], IMAGEDIR)
    files[loc_id] = dl.run()


Starting download for image 20170103_180347_0c41
Starting download for image 20161117_181314_0e26
Starting download for image 20170115_181452_0e19
Starting download for image 20170403_181108_1012
	...waiting
	...waiting
	...waiting
response found.
response found.
response found.
response found.
